In [ ]:
import pandas as pd
import gensim
import numpy as np
import re

In [ ]:
import nltk
import spacy
from nltk.corpus import stopwords

# Download required resources
nltk.download('stopwords')
nltk.download('wordnet')  # For lemmatization
# !python -m spacy download en_core_web_sm  # Install spaCy model

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
df = pd.read_csv("DravidianCodeMix_cleaned.csv")

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.head()

,clean_text
0,enna da ellam avan seyal mari iruku
1,this movei is just like ellam avan seyal
2,padam vanthathum dislike pottavaga yellam yea ...
3,neraya neraya neraya era level thala
4,wow thavala sema mass padam oru pundaikum aagathu


In [ ]:
label_counts = df['ovr_sent'].value_counts()
label_counts

,count
ovr_sent,
1,2643
-1,1718
0,762


In [ ]:
# texts = df['clean_text']
# Labels = df['Label_Enc']

In [ ]:
file_path = '/content/stop_words.txt'

# Initialize an empty list to store stop words
stop_words = []

# Open the file and read lines into the list
with open(file_path, 'r') as file:
    stop_words = [line.strip() for line in file]

# Print the list of stop words to confirm
print(stop_words)

['la', 'is', 'also', 'a', 'இந்த', 'கு', 's', 'vaa', 'for', 'of', 'are', 'and', 'ஆ', 'தான்', 'is', 'm', 'இந்த', 'da', 'di', 'la', 'இந்தப்', 'sir', 've', 'ok', 'a', 'k', 'of', 'la', 'sir', 'என்ன', 'ஆ', 'and', 'ah', 'nxt', 'oru', 'a', 'ah', 'ah', 'ah', 'போல', 'அ', 'அ', 'இல', 'aa', 'back', 'ah', 'to', 'a', 'the', 'for', 'of', 'the', 'in', 'to', 'a', 'is', 'tha', 'aah', 'yaa', 'மிக', 'this', 'is', 'in', 'tha', 'ah', 'ku', 'after', 'sir', 'ennu', 'oru', 'a', 'mam', 'and', 'for', 'laam', 'on', 'of', 'all', 'nu', 'this', 'என்று', 'is', 'a', 'de', 'is', 'pa', 'for', 'my', 'sec', 'is', 'ah', 'a', 'is', 'to', 'in', 'you', 'ku', 'i', 'am', 'from', 'இந்த', 'உம்', 'and', 'very', 'yen', 'oru', 'ella', 'am', 'this', 'would', 'be', 'சூப்பர்', 'once', 'oru', 'oru', 'or', 'is', 'a', 'ena', 'broo', 'na', 'nee', 'from', 'nu', 'of', 'all', 'aga', 'idu', 'nu', 'uhh', 'ula', 'ena', 'da', 'for', 'வே', 'aga', 'i', 'am', 'nee', 'thaan', 'da', 'uh', 'la', 'laa', 'da', 'is', 'tha', 'anne', 'da', 'is', 'tha', 'ah',

In [ ]:
custom_stopwords = [
    'da', 'daa', 'na', 'naa', 'la', 'laa', 'eh', 'ehh', 'eeh', 'ya', 'yaa', 'yah',"ve",
    'nu', 'nuu', 'nd', 'th', 'ku', 'kuu', 'ke', 'keh', 'kee', 'uh', 'uhh', 'uuh',
    'ha', 'haa', 'le', 'leh', 'klam', 'thaan', 'dhaan', 'aa', 'a', 'vaa', 'aah',
    'ah', 'vaa', 'va', 'rr', 'pe', 'ek', 'ye', 'yeh', 'umm', 'um', 'dhaa', 'dha','man','http','https','www','com','amp',
]

In [ ]:
stop_words.extend(custom_stopwords)

In [ ]:
stop_words

['la',
 'is',
 'also',
 'a',
 'இந்த',
 'கு',
 's',
 'vaa',
 'for',
 'of',
 'are',
 'and',
 'ஆ',
 'தான்',
 'is',
 'm',
 'இந்த',
 'da',
 'di',
 'la',
 'இந்தப்',
 'sir',
 've',
 'ok',
 'a',
 'k',
 'of',
 'la',
 'sir',
 'என்ன',
 'ஆ',
 'and',
 'ah',
 'nxt',
 'oru',
 'a',
 'ah',
 'ah',
 'ah',
 'போல',
 'அ',
 'அ',
 'இல',
 'aa',
 'back',
 'ah',
 'to',
 'a',
 'the',
 'for',
 'of',
 'the',
 'in',
 'to',
 'a',
 'is',
 'tha',
 'aah',
 'yaa',
 'மிக',
 'this',
 'is',
 'in',
 'tha',
 'ah',
 'ku',
 'after',
 'sir',
 'ennu',
 'oru',
 'a',
 'mam',
 'and',
 'for',
 'laam',
 'on',
 'of',
 'all',
 'nu',
 'this',
 'என்று',
 'is',
 'a',
 'de',
 'is',
 'pa',
 'for',
 'my',
 'sec',
 'is',
 'ah',
 'a',
 'is',
 'to',
 'in',
 'you',
 'ku',
 'i',
 'am',
 'from',
 'இந்த',
 'உம்',
 'and',
 'very',
 'yen',
 'oru',
 'ella',
 'am',
 'this',
 'would',
 'be',
 'சூப்பர்',
 'once',
 'oru',
 'oru',
 'or',
 'is',
 'a',
 'ena',
 'broo',
 'na',
 'nee',
 'from',
 'nu',
 'of',
 'all',
 'aga',
 'idu',
 'nu',
 'uhh',
 'ula',
 'ena'

In [ ]:
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in custom_stopwords]
    return ' '.join(filtered_words)


In [ ]:
df['clean_text'] = df['clean_text'].apply(remove_stopwords)

In [ ]:
def remove_single_letters_tamil(text):
    """Removes single letters (English and Tamil) from text.

    Args:
      text: The input text string.

    Returns:
      The text string with single letters removed.
    """
    # Define the pattern to match single letters (English and Tamil Unicode range)
    pattern = r'\b(?:[a-zA-Z]|[ऀ-ॿ])\b'

    # Remove the matched single letters
    text = re.sub(pattern, '', text)

    return text

# Apply the function to your 'clean_text' column in the DataFrame
df['clean_text'] = df['clean_text'].apply(remove_single_letters_tamil)

In [ ]:
def process_text_preserve_tamil(text):
    """Removes English stop words, lemmatizes English words, and preserves Tamil words.

    Args:
      text: The input text string.

    Returns:
      The text string with English stop words removed, English words lemmatized, and Tamil words preserved.
    """
    # Tokenize the text
    doc = nlp(text)

    # Process words
    processed_words = []
    for token in doc:
        if token.is_stop and token.lang_ == "en":  # Check if it's an English stop word
            continue  # Skip the stop word
        elif token.is_alpha and token.lang_ == "en":  # Check if it's an English alphabetic word
            processed_words.append(token.lemma_)  # Lemmatize the word
        else:
            processed_words.append(token.text)  # Preserve other words (including Tamil)

    # Join the processed words back into a string
    processed_text = " ".join(processed_words)

    return processed_text

# Apply the function to your 'clean_text' column in the DataFrame
df['clean_text'] = df['clean_text'].apply(process_text_preserve_tamil)

In [ ]:
df=df.drop_duplicates()
df = df[df != ''].reset_index(drop=True)  # Filter out empty strings
df.shape

(40955, 1)

In [ ]:
df.head(20)

,clean_text
0,enna ellam avan seyal mari iruku
1,movei like ellam avan seyal
2,padam vanthathum dislike pottavaga yellam yea ...
3,neraya neraya neraya era level thala
4,wow thavala sema mass padam oru pundaikum aagathu
5,andha unlike panavangaluku kolandha porakathu
6,yaarellam frst like pottutu video paaka start ...
7,ethana padam vanthanu salt pepper mattum .. thãlã
8,thala mass hvy sprt kerala surya anna fan
9,elam avan jayal movie remake pa


In [ ]:
df.to_csv('DravidianCodeMix_cleaned_lemmatized_stop.csv', index=False)

In [ ]:
df1 = pd.read_csv("DravidianCodeMix_cleaned_lemmatized_stop.csv")
df2 = pd.read_csv("Full_data_aspect_removed_words.csv")

In [ ]:
# prompt: i only want clean_text columns on df2

df2 = df2[['clean_text']]

In [ ]:
combined_df = pd.concat([df1[['clean_text']], df2[['clean_text']]], axis=0, ignore_index=True)

# Print some info to check
combined_df.info()
combined_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46076 entries, 0 to 46075
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   clean_text  46075 non-null  object
dtypes: object(1)
memory usage: 360.1+ KB


,clean_text
0,enna ellam avan seyal mari iruku
1,movei like ellam avan seyal
2,padam vanthathum dislike pottavaga yellam yea ...
3,neraya neraya neraya era level thala
4,wow thavala sema mass padam oru pundaikum aagathu


In [ ]:
combined_df = combined_df.dropna(subset=['clean_text'])

# Reset the index after dropping rows
combined_df = combined_df.reset_index(drop=True)

In [ ]:
combined_df.info

<bound method DataFrame.info of                                               clean_text
0                       enna ellam avan seyal mari iruku
1                            movei like ellam avan seyal
2      padam vanthathum dislike pottavaga yellam yea ...
3                   neraya neraya neraya era level thala
4      wow thavala sema mass padam oru pundaikum aagathu
...                                                  ...
46070                  ithu anirudh musi mathiri illayea
46071  ஸூத்தை கிண்டிவிட்டு அதை மணந்து பார்க்கும் இளைஞ...
46072        இதுக்கு பயந்தாடா கங்குவாவ postpone பண்ணீங்க
46073                         trailer onnum worthaa ilaa
46074                        padam vetri pera valthukkal

[46075 rows x 1 columns]>

In [ ]:
combined_df.to_csv('Dravidian_and_own_data_ready_word_embedding.csv', index=False)

In [ ]:
df_demo = df[0:5]
df_demo

,clean_text
0,enna ellam avan seyal mari iruku
1,movei like ellam avan seyal
2,padam vanthathum dislike pottavaga yellam yea ...
3,neraya neraya neraya era level thala
4,wow thavala sema mass padam oru pundaikum aagathu


In [ ]:
df3 = pd.read_csv("Dravidian_and_own_data_ready_word_embedding.csv")

In [ ]:
def create_txt(df,var):
    text=[]
    for i in df[var]:
        li = list(i.split())
        text.append(li)
    return text

In [ ]:
print(create_txt(df_demo,'clean_text'))

[['இரைச்சல்', 'இல்லாத', 'தெளிவான', 'இசை'], ['harris', 'romba', 'naal', 'kalichu', 'super', 'song'], ['hukum', 'pondra', 'iraichal', 'padalai', 'ketu', 'kaathu', 'valichuruchu', 'soft', 'music', 'harris', 'jeyaraj'], ['youtube', 'kuda', 'editing', 'nalla', 'pannuvanga'], ['amaran', 'diwali', 'release']]


In [ ]:
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec

In [ ]:
def word2vec(text,min_count,size,workers,window,sg):
    model = Word2Vec(text,min_count=min_count,vector_size=size,workers=workers,window=window,sg=sg,epochs=150)
    return model

def word_embed(df,num_features,wetype,window,sg):
    text=create_txt(df,'clean_text')

    if(wetype=='w2vec'):
        model=word2vec(text,3,num_features,7,window,sg)
    if(wetype=='ft'):
        model = FastText(text,vector_size=num_features,window=window,sg=sg,epochs=150)

    return model

In [ ]:
cbow=word_embed(df3,300,'w2vec',2,0)
cbow.wv.save_word2vec_format('w2v_cbow_300_fyp_.txt', binary=False)


In [ ]:
skip=word_embed(df3,300,'w2vec',2,1)
skip.wv.save_word2vec_format('w2v_skip_300_fyp_window2.txt', binary=False)


In [ ]:
ft=word_embed(df3,300,'ft',2,1)
ft.wv.save_word2vec_format('ft_300_fyp_5k_150epoch_window5.txt', binary=False)